# Back to Pandas and the csv#

**Read csv and inspect result**

In [215]:
import pandas
import re

In [216]:
data = pandas.DataFrame.from_csv("enhanced.ledger.sdi.log", sep="|", index_col=None)
data.describe()

,ReportingEntity,GLBalance,GLBalanceInReportingCCY
count,0.0,1.000000e+02,1.000000e+02
mean,NaN,2.409708e+04,1.123240e+05
std,NaN,6.217487e+05,1.458348e+06
min,NaN,-1.962756e+06,-3.163277e+06
25%,NaN,-2.546282e+05,-2.891908e+05
50%,NaN,-3.894455e+04,-3.882858e+04
75%,NaN,2.256189e+05,2.826291e+05
max,NaN,2.070709e+06,1.105048e+07


In [217]:
data.head()

,RecordType,SourceSystemId,LegalEntity,ReportingEntity,CostCentre,GLKey,GLBalanceCurrencyCode,GLBalance,BankingORTradingBook,GroupReconciliationKey,GLBalanceInReportingCCY,ReportingBalanceCurrencyCode
0,B,HUBAU,'4435',NaN,AUHBAP,06764-A,AUD,326172.28,T,AO27875,367922.137,AUD
1,B,HUBAU,'4435',NaN,AUHBAP,03404-A,AUD,-999.99,T,AS40306,-950.623,AUD
2,B,HUBAU,NaN,NaN,AUHBAP,06413-L,AUD,-251780.78,T,ZZ41246,-266514.437,AUD
3,B,HUBAU,'4435',NaN,AUHBAP,08116-L,AUD,-43457.35,T,WT99771,-49510.600,AUD
4,B,HUBAU,'4435',NaN,AUHBAP,00690-A,CNY,235924.21,T,HC43842,1236167.478,AUD


In [218]:
data_raw = pandas.DataFrame.from_csv("ledger.sdi.log", header=None, sep="|", index_col=None)
data_raw.describe()

,3,7,10
count,0.0,1.000000e+02,1.000000e+02
mean,NaN,2.409708e+04,1.123240e+05
std,NaN,6.217487e+05,1.458348e+06
min,NaN,-1.962756e+06,-3.163277e+06
25%,NaN,-2.546282e+05,-2.891908e+05
50%,NaN,-3.894455e+04,-3.882858e+04
75%,NaN,2.256189e+05,2.826291e+05
max,NaN,2.070709e+06,1.105048e+07


In [219]:
data.RecordType.astype('category').describe()

count     97
unique     2
top        B
freq      96
Name: RecordType, dtype: object

In [220]:
import pandas_profiling
pandas_profiling.ProfileReport(data)

/Users/simonshapiro/miniconda3/envs/dataGen/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}


Number of variables,12
Number of observations,100
Total Missing (%),10.9%
Total size in memory,9.5 KiB
Average record size in memory,96.8 B
Numeric,2
Categorical,9
Date,0
Text (Unique),0
Rejected,1
Distinct count,3


In [221]:
def CustomRule(rule_id:str, rule_desc:str, fn):
    def rule(row, selector:str, parameters, id_by=None):
        passed = fn(row, selector, parameters)
        res = {
            "RuleID": rule_id,
            "RuleDescription": rule_desc,
            "AttributeTested": selector,
            "ValueTested": row[selector],        
            "Result": "Passed" if passed else "Failed",
        }
        if id_by == None:
            res.update(row.to_dict())
        else:
            res.update(row[id_by].to_dict())
        return res
    return rule

def Rule_001(row, selector:str, parameters, id_by=None):
    rule_id = 'Rule_001'
    rule_desc = 'Missing value'
    passed = not pandas.isnull(row[selector])
    res = {
        "RuleID": rule_id,
        "RuleDescription": rule_desc,
        "AttributeTested": selector,
        "ValueTested": row[selector],        
        "Result": "Passed" if passed else "Failed",
    }
    if id_by == None:
        res.update(row.to_dict())
    else:
        res.update(row[id_by].to_dict())
    return res

def Rule_002(row, selector:str, parameters, id_by=None):
    rule_id = 'Rule_002'
    rule_desc = 'Value not in list'
    passed = row[selector] in parameters["list"]
    res = {
        "RuleID": rule_id,
        "RuleDescription": rule_desc,
        "AttributeTested": selector,
        "ValueTested": row[selector],        
        "Result": "Passed" if passed else "Failed",
    }
    if id_by == None:
        res.update(row.to_dict())
    else:
        res.update(row[id_by].to_dict())
    return res

def Rule_003(row, selector:str, parameters, id_by:list=None):
    rule_id = 'Rule_003'
    rule_desc = 'Pattern mismatch'
    #Set up and test regex
    pattern = parameters["pattern"]
    r = re.compile(pattern)
    result = "Passed" if r.match(str(row[selector])) else "Failed"
    res = {
        "RuleID": rule_id,
        "RuleDescription": rule_desc,
        "AttributeTested": selector,
        "ValueTested": str(row[selector]),        
        "Result": result,
    }
    if id_by == None:
        res.update(row.to_dict())
    else:
        res.update(row[id_by].to_dict())
    return res



def test_row(row:pandas.DataFrame, rule_set, id_by:list=None, verbose=False):
    results = []
    for attr, attr_rules in rule_set.items():
        for rrule in attr_rules:
            rule = rrule["rule"]
            results.append(rule(row, attr, rrule["parms"], id_by))
#    print(results)
    results = list(filter(lambda r: r["Result"] == "Failed", results)) if not verbose else results
    return results

In [222]:
def test_fn(row, selector, parameters):
    pattern = parameters["pattern"]
    r = re.compile(pattern)
    result = True if r.match(str(row[selector])) else False
    return result

CR_001 = CustomRule("CR_001", "Customer version of pattern via re", test_fn)

def lookup_in_csv(row, selector, parameters):
    # Sets up a data frame for a csv in order to fake a database table
    # !!!! Highly inefficient because it should be an api call to a refdata service
    file_name = parameters["file"]
    data = pandas.DataFrame.from_csv(file_name, sep=",", index_col=None)
    search_col = parameters["col"]
    if not pandas.isnull(row[selector]):
        passed = row[selector] in list(data[search_col])
    else: passed = False
    return passed

CR_002 = CustomRule("CR_002", "Test against reference csv", lookup_in_csv)

def test_balance_conforms_to_GLKey(row, selector, parameters):
    n = row[selector]
    gl_key = row["GLKey"]
    if not pandas.isnull(gl_key):
        last_char = gl_key[-1]
        if (last_char == 'L') and (n <= 0.0):
            result = True
        elif (last_char == "A") and (n >= 0.0):
            result = True
        else: result = False
    else: result = False
    return result
    
CR_003 = CustomRule("CR_003", "Test for sign of balance in account", test_balance_conforms_to_GLKey)

rule_set = {
    "RecordType": [
        {"rule":Rule_001, "parms":{}},
        {"rule":Rule_002, "parms":{"list":["B"]}},
      ],  
    "SourceSystemId": [
        {"rule":Rule_001, "parms":{}},
        {"rule":Rule_002, "parms":{"list":["HUBAU"]}},
      ],  
    "LegalEntity": [
        {"rule":Rule_001, "parms":{}},
        {"rule":Rule_002, "parms":{"list":["'4435'"]}},  #This should be a call to an authatative source
        {"rule":CR_002, "parms":{"file":"company_codes.csv",
                                "col": "SaracenCode"}},  #This should be a call to an authatative source
      ],
    "GLKey": [
        {"rule":Rule_001, "parms":{}},
        {"rule":Rule_003, "parms": {"pattern": "[0-9]{5}-[A,L]"}}
    ],
    "GLBalance": [
        {"rule": CR_003, "parms": {}} #Number sign should follow last character of GLKey
    ],
    "BankingORTradingBook": [
        {"rule":Rule_001, "parms":{}},
        {"rule":Rule_002, "parms":{"list":["B", "T"]}},
        ],
    "GroupReconciliationKey": [
        {"rule":Rule_001, "parms":{}},
        {"rule":Rule_003, "parms":{"pattern":"[A-Z]{2}[0-9]{5}"}},
#        {"rule":CR_001, "parms":{"pattern":"[A-Z]{2}[0-9]{5}"}},
    ],
    "GLBalanceCurrencyCode": [
        {"rule":Rule_001, "parms":{}},
        {"rule":CR_002, "parms": {
                                "file": "currency_codes.csv",
                                "col": "Alphabetic Code"
                                }}
    ],
    "ReportingBalanceCurrencyCode": [
        {"rule":Rule_001, "parms":{}},
        {"rule":CR_002, "parms": {
                                "file": "currency_codes.csv",
                                "col": "Alphabetic Code"
                                }}
    ]

}

file_results = []
for i, row in data.iterrows():
#    row_results = test_row(row, rule_set, id_by=["GLKey", "LegalEntity"], verbose=True)

    row_results = test_row(row, rule_set, verbose=False)

    if len(row_results) > 0: 
        for row in row_results:
            file_results.append(row)
dq_detail_results = pandas.DataFrame.from_records(file_results)
#dq_detail_results[["GroupReconciliationKey", "Result", "RuleDescription", "AttributeTested", "ValueTested"]]
dq_detail_results.sort_values("AttributeTested")

,AttributeTested,BankingORTradingBook,CostCentre,GLBalance,GLBalanceCurrencyCode,GLBalanceInReportingCCY,GLKey,GroupReconciliationKey,LegalEntity,RecordType,ReportingBalanceCurrencyCode,ReportingEntity,Result,RuleDescription,RuleID,SourceSystemId,ValueTested
78,BankingORTradingBook,NaN,AUHBAP,125909.03,AUD,147072.021,02523-A,CM95739,'4435',B,AUD,NaN,Failed,Value not in list,Rule_002,NaN,NaN
55,BankingORTradingBook,NaN,AUHBAP,658709.44,AUD,599259.111,08702-A,UU68287,'4435',B,AUD,NaN,Failed,Value not in list,Rule_002,HUBAU,NaN
54,BankingORTradingBook,NaN,AUHBAP,658709.44,AUD,599259.111,08702-A,UU68287,'4435',B,AUD,NaN,Failed,Missing value,Rule_001,HUBAU,NaN
45,BankingORTradingBook,NaN,AUHBAP,984284.41,AUD,1109204.057,01312-A,JL65272,'4435',B,AUD,NaN,Failed,Value not in list,Rule_002,HUBAU,NaN
44,BankingORTradingBook,NaN,AUHBAP,984284.41,AUD,1109204.057,01312-A,JL65272,'4435',B,AUD,NaN,Failed,Missing value,Rule_001,HUBAU,NaN
77,BankingORTradingBook,NaN,AUHBAP,125909.03,AUD,147072.021,02523-A,CM95739,'4435',B,AUD,NaN,Failed,Missing value,Rule_001,NaN,NaN
28,BankingORTradingBook,NaN,AUHBAP,-83447.89,AUD,-96991.697,07792-L,UR27277,'4435',B,AUD,NaN,Failed,Value not in list,Rule_002,HUBAU,NaN
27,BankingORTradingBook,NaN,AUHBAP,-83447.89,AUD,-96991.697,07792-L,UR27277,'4435',B,AUD,NaN,Failed,Missing value,Rule_001,HUBAU,NaN
26,BankingORTradingBook,NaN,AUHBAP,-101629.53,SGD,-102720.523,01237-L,XA44145,'4435',B,AUD,NaN,Failed,Value not in list,Rule_002,HUBAU,NaN
25,BankingORTradingBook,NaN,AUHBAP,-101629.53,SGD,-102720.523,01237-L,XA44145,'4435',B,AUD,NaN,Failed,Missing value,Rule_001,HUBAU,NaN


In [223]:
pandas.pivot_table(
    dq_detail_results,
    values=["RuleID"],
    index=["RuleDescription", "Result"],
    columns=["AttributeTested"],
    aggfunc='count')

RuleID            \
AttributeTested                            BankingORTradingBook GLBalance   
RuleDescription                     Result                                  
Missing value                       Failed                  5.0       NaN   
Pattern mismatch                    Failed                  NaN       NaN   
Test against reference csv          Failed                  NaN       NaN   
Test for sign of balance in account Failed                  NaN       7.0   
Value not in list                   Failed                  5.0       NaN   

                                                                        \
AttributeTested                            GLBalanceCurrencyCode GLKey   
RuleDescription                     Result                               
Missing value                       Failed                   3.0   3.0   
Pattern mismatch                    Failed                   NaN   5.0   
Test against reference csv          Failed                   5.0   NaN   
Test for sign of balance in account Failed                   NaN   NaN   
Value not in list                   Failed                   NaN   NaN   

                                                                               \
AttributeTested                            GroupReconciliationKey LegalEntity   
RuleDescription                     Result                                      
Missing value                       Failed                    4.0         5.0   
Pattern mismatch                    Failed                    5.0         NaN   
Test against reference csv          Failed                    NaN         5.0   
Test for sign of balance in account Failed                    NaN         NaN   
Value not in list                   Failed                    NaN         5.0   

                                                       \
AttributeTested                            RecordType   
RuleDescription                     Result              
Missing value                       Failed        3.0   
Pattern mismatch                    Failed        NaN   
Test against reference csv          Failed        NaN   
Test for sign of balance in account Failed        NaN   
Value not in list                   Failed        4.0   

                                                                         \
AttributeTested                            ReportingBalanceCurrencyCode   
RuleDescription                     Result                                
Missing value                       Failed                          3.0   
Pattern mismatch                    Failed                          NaN   
Test against reference csv          Failed                          5.0   
Test for sign of balance in account Failed                          NaN   
Value not in list                   Failed                          NaN   

                                                           
AttributeTested                            SourceSystemId  
RuleDescription                     Result                 
Missing value                       Failed            2.0  
Pattern mismatch                    Failed            NaN  
Test against reference csv          Failed            NaN  
Test for sign of balance in account Failed            NaN  
Value not in list                   Failed            5.0